# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
# check gpu type
!nvidia-smi

Thu Mar  2 03:54:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download data
If the Google Drive links below do not work, you can use the dropbox link below or download data from [Kaggle](https://www.kaggle.com/competitions/ml2023spring-hw1/overview), and upload data manually to the workspace.

In [2]:
# google drive link
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

--2023-03-02 03:54:36--  https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lmy1riadzoy0ahw/covid.train.csv [following]
--2023-03-02 03:54:37--  https://www.dropbox.com/s/raw/lmy1riadzoy0ahw/covid.train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9cdcf06526902c9e8d76ab48aa.dl.dropboxusercontent.com/cd/0/inline/B3cgsG6rXGmDIbYZwbe4GCRLLSSBtU63tzkMJIidmDbaP2MtU16FaCvsGBKvQ1ZwAKBbEVAVfW5rLBhSifTcuR7MWKl0ghY1I-5Bb4LyxDJVGYITfv4g9vehcwMpyrIFu8QUD5PyvISA9j9YubLpoiwsc8FWVzREsaKAKy2FW4ULzQ/file# [following]
--2023-03-02 03:54:37--  https://uc9cdcf06526902c9e8d76ab48aa.dl.dropboxusercontent.com/cd/0/inline/B3cgsG6rXGmDIbYZwbe4GCRLLSSBtU63tzkMJIidmDbaP2MtU16FaCvsGBKvQ1Z

# Import packages

In [3]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [5]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [6]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            

            # nn.Linear(input_dim, 500),
            # nn.ReLU(),
            # nn.Linear(500, 250),
            # nn.ReLU(),
            # # nn.Linear(input_dim, 32),
            # # nn.ReLU(),
            # nn.Linear(250,100),
            # nn.ReLU(),
            # nn.Linear(100, 1),
            # nn.ReLU()

            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8,1),
            nn.ReLU()
            

            # nn.Linear(input_dim, 32),
            # # nn.ReLU(),
            # nn.SiLU(),
            # nn.Linear(32, 16),
            # # nn.ReLU(),
            # nn.SiLU(),
            # nn.Linear(16,1),
            # # nn.ReLU()
            # nn.SiLU()

            # nn.Linear(input_dim, 16),
            # nn.ReLU(),
            # nn.Linear(16, 8),
            # nn.ReLU(),
            # nn.Linear(8,4),
            # nn.ReLU(),
            # nn.Linear(4, 1),
            # nn.ReLU()

            # nn.Linear(input_dim, 160),
            # nn.ReLU(),
            # nn.Linear(160, 80),
            # nn.ReLU(),
            # nn.Linear(80,40),
            # nn.ReLU(),
            # nn.Linear(40, 1),
            # nn.ReLU()

            # nn.Linear(input_dim, 1000),
            # nn.ReLU(),
            # nn.Linear(1000, 400),
            # nn.ReLU(),
            # nn.Linear(150, 100),
            # nn.ReLU(),
            # nn.Linear(400, 100),
            # nn.ReLU(),
            # nn.Linear(100, 50),
            # nn.ReLU(),
            # nn.Linear(50,1)

            # nn.Linear(input_dim, 1024),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(1024, 750),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(750, 512),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(512, 256),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(256, 128),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(128, 64),
            # nn.ReLU(),
            # # nn.SiLU(),
            # # nn.ELU(),
            # nn.Linear(64,1),
            
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [7]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # TODO: Select suitable feature columns.
        feat_idx = [35,36,37,47,48,52,53,54,55,65,66,70,71,72,73,83,84]

        # feat_idx = [35,36,37,47,48,53,54,55,65,66,71,72,73,83,84]

        # feat_idx = [47,48,52,65,66,70,83]
        
        #42 is large_indoor_event        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [8]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).

    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.025, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 200, gamma=0.1, last_epoch=- 1, verbose=False)

    # optimizer = torch.optim.ASGD(model.parameters(), lr=0.01, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0, foreach=None, maximize=False)

    # optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0, foreach=None, maximize=False)

    

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=False) #this is the progress bar

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        scheduler.step() #change the lr dynamically

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 548769,      # Your seed number, you can pick your lucky number. :)
    
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 2500,     # Number of epochs.            
    'batch_size': 24, 
    'learning_rate': 1e-4,              
    'early_stop': 690,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [10]:
same_seed(config['seed'])
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
number of features: 17


# Start training!

In [11]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/2500]: Train loss: 23.5435, Valid loss: 4.9953
Saving model with loss 4.995...


Epoch [2/2500]: Train loss: 2.1458, Valid loss: 1.3030
Saving model with loss 1.303...


Epoch [3/2500]: Train loss: 1.4238, Valid loss: 1.3356


Epoch [4/2500]: Train loss: 1.3285, Valid loss: 2.7971


Epoch [5/2500]: Train loss: 1.4708, Valid loss: 1.3773


...

Epoch [47/2500]: Train loss: 1.1393, Valid loss: 1.6698


Epoch [48/2500]: Train loss: 1.3354, Valid loss: 0.9815


Epoch [49/2500]: Train loss: 1.1916, Valid loss: 1.8262


Epoch [50/2500]: Train loss: 1.2012, Valid loss: 2.0228


KeyboardInterrupt: ignored

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')

# Download

Run this block to download the `pred.csv` automatically.

In [ ]:
from google.colab import files
files.download('pred.csv')

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)